In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms

import os

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [28]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()

        self.conv_block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False), # kernel_size=3, padding=1
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False), # kernel_size=3, padding=1
            nn.BatchNorm2d(out_channels)
        )

        self.relu = nn.ReLU(inplace=True)

        self.downsample = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
            nn.BatchNorm2d(out_channels)
        ) if stride != 1 or in_channels != out_channels else None

    def forward(self, x):
        residual = x
        out = self.conv_block(x)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out


In [29]:
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        #self.in_channels = 64
        self.in_channels = 128
        channels = 128
        self.initial = nn.Sequential(
            nn.Conv2d(3, channels, kernel_size=3, stride=1, padding=1, bias=False),
            nn.BatchNorm2d(channels),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        )
        self.layer1 = self.make_layer(block, channels, layers[0])
        self.layer2 = self.make_layer(block, channels * 2, layers[1], stride=2)
        self.layer3 = self.make_layer(block, channels * 4, layers[2], stride=2)
        self.layer4 = self.make_layer(block, channels * 8, layers[3], stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))

        self.dropout = nn.Dropout(0.5)

        self.fc = nn.Linear(channels * 8, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.initial(x)
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)

        x = self.dropout(x)

        x = self.fc(x)
        return x

In [34]:
def find_max_index(predictions, target):
    predicted_index = predictions.index(max(predictions))
    return predicted_index == target

def train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, epochs, save_path):
    for epoch in range(epochs):
        model.train()
        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

        with torch.no_grad():
            model.eval()
            test_loss = 0.0
            acc = 0
            total = 0

            for data, target in test_loader:
                data, target = data.to(device), target.to(device)
                output = model(data)
                output_list = output.cpu().detach().numpy().tolist()
                target_list = target.cpu().detach().numpy().tolist()

                for i in range(len(data)):
                    if find_max_index(output_list[i], target_list[i]):
                        acc += 1

                total += len(data)
                test_loss += criterion(output, target).item()

            average_loss = test_loss / len(test_loader)
            accuracy = acc / total

            print(f'Epoch: {epoch+1} | Train loss: {loss.item():.4f} | Test loss: {average_loss:.4f} | Accuracy: {accuracy:.4f}')
            
            # Save model
            if (epoch % 3 == 0):
                torch.save(model.state_dict(), os.path.join(save_path, f'model_epoch_{epoch+1}.pth'))


In [35]:
# Hyperparameters
batch = 128
lr = 0.001
epochs = 35

save_path = './saved_models_2'
os.makedirs(save_path, exist_ok=True)

In [38]:
# Data loading and preprocessing
transform = transforms.Compose([
    #transforms.RandomCrop(32, padding=1),
    transforms.RandomHorizontalFlip(p=0.5),
    # transforms.RandomVerticalFlip(p=0.5),
    # transforms.RandomRotation(degrees=15),
    # transforms.RandomResizedCrop(size=32, scale=(0.8, 1.0)),
    # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    # transforms.RandomGrayscale(p=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.4915, 0.4823, 0.4468), (0.2023, 0.2435, 0.2616))

    # transforms.ToTensor(),
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    # transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))
])

train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch, shuffle=True)

test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [39]:
# Model, loss function, and optimizer
model = ResNet(ResidualBlock, [2, 2, 2, 2]).to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)

print("Результаты:")
# Training and evaluation
train_and_evaluate(model, train_loader, test_loader, criterion, optimizer, epochs, save_path)

Результаты:
Epoch: 1 | Train loss: 0.9808 | Test loss: 1.2093 | Accuracy: 0.5696
Epoch: 2 | Train loss: 0.6937 | Test loss: 0.9286 | Accuracy: 0.6773
Epoch: 3 | Train loss: 0.4644 | Test loss: 0.7555 | Accuracy: 0.7445
Epoch: 4 | Train loss: 0.6155 | Test loss: 0.6593 | Accuracy: 0.7777
Epoch: 5 | Train loss: 0.3704 | Test loss: 0.5664 | Accuracy: 0.8099
Epoch: 6 | Train loss: 0.5049 | Test loss: 0.5392 | Accuracy: 0.8224
Epoch: 7 | Train loss: 0.4410 | Test loss: 0.5287 | Accuracy: 0.8287
Epoch: 8 | Train loss: 0.3442 | Test loss: 0.4780 | Accuracy: 0.8368
Epoch: 9 | Train loss: 0.2013 | Test loss: 0.4578 | Accuracy: 0.8493
Epoch: 10 | Train loss: 0.2716 | Test loss: 0.4261 | Accuracy: 0.8643
Epoch: 11 | Train loss: 0.2090 | Test loss: 0.5293 | Accuracy: 0.8372
Epoch: 12 | Train loss: 0.1660 | Test loss: 0.4400 | Accuracy: 0.8610
Epoch: 13 | Train loss: 0.1107 | Test loss: 0.4949 | Accuracy: 0.8591
Epoch: 14 | Train loss: 0.1311 | Test loss: 0.5196 | Accuracy: 0.8589
Epoch: 15 | Train

In [1]:
from sklearn.metrics import precision_score

# Функция для расчета precision
def evaluate_precision(model, test_loader, criterion):
    model.eval()
    y_true = []
    y_pred = []

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            y_true.extend(target.cpu().numpy())
            y_pred.extend(predicted.cpu().numpy())

    precision = precision_score(y_true, y_pred, average='macro')
    return precision

# Загрузка сохраненной модели
def load_model(model, checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    model.load_state_dict(checkpoint['state_dict'])


# Загрузка сохраненной модели
checkpoint_path = os.path.join('./saved_models_2', 'model_epoch_34.pth') 
load_model(model, checkpoint_path)

# Оценка точности
precision = evaluate_precision(model, test_loader, nn.CrossEntropyLoss())
print(f'Precision: {precision:.4f}')

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


NameError: name 'os' is not defined